In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from eofs.xarray import Eof
import time

# Read data

Load the data into an xarray dataset.

In [ ]:
dataset = xr.open_dataset('/home/disk/eos12/wycheng/data/US/dataset.nc',chunks={'Time':5848, 'lev':'auto', 'lat':'auto', 'lon':'auto'})

Assign a new variable "isT" based on whether there is lightning stroke observed.

In [ ]:
dataset = dataset.assign(isT=lambda dataset: 1.0*(dataset.F>0))

Select a sub-domain centering around west coast region.

In [ ]:
dataset = dataset.isel(lat=slice(10,20), lon=slice(15,25))

# Feature Engineering

We use EOF (PCA) to reconstruct the 1-D data (profiles of temperature and moisture profiles) and 2-D data (images of geopotential height) into new features to feed into the ML model.

In [ ]:
t_stacked = dataset['t'].stack(time=('Time','lat','lon')).transpose("time", "lev")
q_stacked = dataset['q'].stack(time=('Time','lat','lon')).transpose("time", "lev")

In [ ]:
start_time = time.time()
t1d_solver = Eof(t_stacked,center='True')
q1d_solver = Eof(q_stacked,center='True')
print("%s seconds" % (time.time() - start_time))

In [ ]:
t_pcs = t1d_solver.projectField(t_stacked, eofscaling=1, neofs=10)
q_pcs = q1d_solver.projectField(q_stacked, eofscaling=1, neofs=10)

In [ ]:
t_pcs = t_pcs.unstack().transpose("Time", "mode", "lat", "lon")
q_pcs = q_pcs.unstack().transpose("Time", "mode", "lat", "lon")

In [ ]:
for imode in range(10):
    exec( 'dataset = dataset.assign(tpc'+str(imode)+'=t_pcs[:,imode,:,:])' )
    exec( 'dataset = dataset.assign(qpc'+str(imode)+'=q_pcs[:,imode,:,:])' )

In [ ]:
dataset

In [ ]:
t1d_ve = t1d_solver.varianceFraction(neigs=10)
q1d_ve = q1d_solver.varianceFraction(neigs=10)

In [ ]:
plt.plot(np.linspace(1,10,10),t1d_ve,'b-',label='t');
plt.plot(np.linspace(1,10,10),np.cumsum(t1d_ve),'b--')
plt.plot(np.linspace(1,10,10),q1d_ve,'g-',label='q');
plt.plot(np.linspace(1,10,10),np.cumsum(q1d_ve),'g--')
plt.legend()
plt.xlabel('PC #')
plt.ylabel('Fraction of variance explained')
plt.xlim([1,10])
plt.ylim([0,1])

In [ ]:
t1d_eofs = t1d_solver.eofs(eofscaling=2, neofs=10)
q1d_eofs = q1d_solver.eofs(eofscaling=2, neofs=10)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.plot(t1d_eofs[0,:],np.linspace(1000,100,10),'b-',label='EOF1')
ax1.plot(t1d_eofs[1,:],np.linspace(1000,100,10),'b--',label='EOF2')
ax1.plot(t1d_eofs[2,:],np.linspace(1000,100,10),'b:',label='EOF3')

ax2.plot(q1d_eofs[0,:],np.linspace(1000,100,10),'g-',label='EOF1')
ax2.plot(q1d_eofs[1,:],np.linspace(1000,100,10),'g--',label='EOF2')
ax2.plot(q1d_eofs[2,:],np.linspace(1000,100,10),'g:',label='EOF3')

ax1.set_ylabel('pressure (hPa)')
ax1.set_xlabel('T\' (K)')
ax1.legend()

ax2.set_xlabel('q\' (kg/kg)')
ax2.legend()

plt.gca().invert_yaxis()

In [ ]:
for ilev in range(10):
    exec( 'dataset = dataset.assign(t'+str(ilev)+'=dataset.t[:,ilev,:,:])' )
    exec( 'dataset = dataset.assign(q'+str(ilev)+'=dataset.q[:,ilev,:,:])' )

In [ ]:
dataframe = dataset.where( (dataset.island == 1) ).to_dataframe().dropna(axis=0)

In [ ]:
dataframe

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Hierarchy - 1: Predicting power for CAPE

### Using all levels of T and q directly

In [ ]:
feature_name  = ['t0','t1','t2','t3','t4','t5','t6','t7','t8','t9','q0','q1','q2','q3','q4','q5','q6','q7','q8','q9']
output_name   = ['cape']
X = dataframe[feature_name]
y = dataframe[output_name]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=None)

In [ ]:
rfreg = RandomForestRegressor(n_estimators=4,
                              max_depth=4,
                              random_state=0,
                              n_jobs=4,
                              verbose=3)

In [ ]:
start_time = time.time()
rfreg.fit(X_train[feature_name], y_train[output_name].values.ravel())
print("%s seconds" % (time.time() - start_time))

In [ ]:
y_predict_rfreg = rfreg.predict(X_test[feature_name])

In [ ]:
y_predict_truth = y_test[output_name].values.ravel()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_predict_rfreg,y_predict_truth)
ax.set_title('R2: ' + str(r2_score(y_predict_rfreg,y_predict_truth)))
ax.set_xlabel('CAPE (RF regressor)')
ax.set_ylabel('CAPE (ground truth)')

In [ ]:
train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(rfreg, X, y, cv=4, n_jobs=4,
                       train_sizes=np.linspace(0.1,1.0,4),
                       return_times=True,verbose=3)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
fit_times_mean = np.mean(fit_times, axis=1)
fit_times_std = np.std(fit_times, axis=1)

axes.grid()
axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                      train_scores_mean + train_scores_std, alpha=0.1,
                      color="r")
axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                      test_scores_mean + test_scores_std, alpha=0.1,
                      color="g")
axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
              label="Training score")
axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
              label="Cross-validation score")
axes.legend(loc="best")

### Using EOFs of T, q profiles

In [ ]:
#feature_name  = ['tpc0','tpc1','tpc2','tpc3','tpc4','tpc5','tpc6','tpc7','tpc8','tpc9','qpc0','qpc1','qpc2','qpc3','qpc4','qpc5','qpc6','qpc7','qpc8','qpc9']
feature_name  = ['tpc0','tpc1','tpc2','tpc3','tpc4','qpc0','qpc1','qpc2','qpc3','qpc4']
#feature_name  = ['tpc0','tpc1','tpc2','qpc0','qpc1','qpc2']
#feature_name  = ['tpc0','tpc1','qpc0','qpc1']
#feature_name  = ['tpc0','qpc0']
output_name   = ['cape']
X = dataframe[feature_name]
y = dataframe[output_name]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=None)

In [ ]:
rfreg = RandomForestRegressor(n_estimators=4,
                              max_depth=4,
                              random_state=0,
                              n_jobs=4,
                              verbose=3)

In [ ]:
start_time = time.time()
rfreg.fit(X_train[feature_name], y_train[output_name].values.ravel())
print("%s seconds" % (time.time() - start_time))

In [ ]:
y_predict_rfreg = rfreg.predict(X_test[feature_name])

In [ ]:
y_predict_truth = y_test[output_name].values.ravel()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_predict_rfreg,y_predict_truth)
ax.set_title('R2: ' + str(r2_score(y_predict_rfreg,y_predict_truth)))
ax.set_xlabel('CAPE (RF regressor)')
ax.set_ylabel('CAPE (ground truth)')

In [ ]:
train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(rfreg, X, y, cv=4, n_jobs=4,
                       train_sizes=np.linspace(0.1,1.0,4),
                       return_times=True,verbose=3)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
fit_times_mean = np.mean(fit_times, axis=1)
fit_times_std = np.std(fit_times, axis=1)

axes.grid()
axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                      train_scores_mean + train_scores_std, alpha=0.1,
                      color="r")
axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                      test_scores_mean + test_scores_std, alpha=0.1,
                      color="g")
axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
              label="Training score")
axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
              label="Cross-validation score")
axes.legend(loc="best")

## Hierarchy - 2: Predicting power for isT

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, precision_score, average_precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import auc

In [ ]:
n_models       = 6
feature_name0  = ['pcp','cape']
feature_name1  = ['pcp','cape']
feature_name2  = ['pcp','t0','t1','t2','t3','t4','t5','t6','t7','t8','t9','q0','q1','q2','q3','q4','q5','q6','q7','q8','q9']
feature_name3  = ['pcp','tpc0','tpc1','qpc0','qpc1']
feature_name4  = ['pcp','tpc0','tpc1','tpc2','qpc0','qpc1','qpc2']
feature_name5  = ['pcp','cape','tpc0','tpc1','tpc2','qpc0','qpc1','qpc2']
output_name    = ['isT']


In [ ]:
X = dataframe.drop(output_name,axis=1)
y = dataframe[output_name] 
                   
X_train_raw, X_test, y_train_raw, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
X_train, y_train = undersample.fit_resample(X_train_raw, y_train_raw)

y_predict_truth = y_test[output_name].values.ravel()

### R14

In [ ]:
import scipy as sp
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import normalize

In [ ]:
class R14:
    
    def fit(CAPE,pcp,y):

        thrs = sp.optimize.fminbound(lambda x: -f1_score(y, ((CAPE*pcp > x) * 1.0).astype(int)), 0, 4000)
        fval = f1_score(y, ((CAPE*pcp >= thrs) * 1.0).astype(int))
        
        return thrs, fval
    
    def predict(CAPE,pcp,thrs):
        
        y_predict = ((CAPE*pcp >= thrs) * 1.0).astype(int)
        y_predict_proba = CAPE*pcp
        
        return y_predict, y_predict_proba/np.max(y_predict_proba)

In [ ]:
[r14_thrs,fval] = R14.fit(X_train['cape'],X_train['pcp'],y_train)

In [ ]:
y_predict_r14, y_score0 = R14.predict(X_test['cape'],X_test['pcp'],r14_thrs)

In [ ]:
AUROCC0 = roc_auc_score(y_predict_truth, y_score0)

precision, recall, thresholds = precision_recall_curve(y_predict_truth, y_score0)
AUPRC0  = auc(recall, precision)

### RFC

In [ ]:
for imodel in np.arange(1,n_models,1):
    
    exec( 'rfclf'+str(imodel)+' = RandomForestClassifier(n_estimators=10, '\
                               'max_depth=4,'\
                               'min_samples_split=10, '\
                               'random_state=0)' )
    
    exec( 'rfclf'+str(imodel)+'.fit(X_train[feature_name'+str(imodel)+'], y_train[output_name].values.ravel())' )
    
    exec( 'y_predict_rfclf'+str(imodel)+' = rfclf'+str(imodel)+'.predict(X_test[feature_name'+str(imodel)+'])' )
    
    exec( 'y_score'+str(imodel)+' = rfclf'+str(imodel)+'.predict_proba(X_test[feature_name'+str(imodel)+'])[:,1]' )
    exec( 'precision, recall, thresholds = precision_recall_curve(y_predict_truth, y_score'+str(imodel)+')' )
    
    exec( 'AUROCC'+str(imodel)+' = roc_auc_score(y_predict_truth, y_score'+str(imodel)+')' )
    exec( 'AUPRC'+str(imodel)+' = auc(recall, precision)' )

In [ ]:
colors  = ['k','b','orange','g','r','purple']
fig, ax = plt.subplots()

for imodel in np.arange(0,n_models,1):
    exec( 'fpr, tpr, threshold = roc_curve(y_predict_truth, y_score'+str(imodel)+')' )
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, c=colors[imodel], label = 'Model '+str(imodel)+' (AUC = %0.2f)'% roc_auc)
    
plt.xlabel('False Alarm Rate')
plt.ylabel('True Positive Rate')
plt.legend(fontsize=12,loc='best')
plt.show()  

In [ ]:
fig, ax = plt.subplots()

for imodel in np.arange(0,n_models,1):
    exec( 'precision, recall, thresholds = precision_recall_curve(y_predict_truth, y_score'+str(imodel)+')' )
    pr_auc = auc(recall, precision)
    ax.plot(precision, recall, c=colors[imodel], label = 'Model '+str(imodel)+' (AUC = %0.2f)'% pr_auc)
    
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(fontsize=12,loc='best')
plt.show()  

In [ ]:
markers = ['.','v','s','p','*','x','d']
fig, ax = plt.subplots()
for imodel in np.arange(0,n_models,1):
    exec( 'ax.scatter(AUPRC'+str(imodel)+',AUROCC'+str(imodel)+', c=colors[imodel], marker=\''+markers[imodel]+'\',label=\'Model '+str(imodel)+'\')' )

ax.set_title('Model skill')
ax.set_xlabel('Area under PR curve')
ax.set_ylabel('Area under ROC curve')
#ax.set_xlim([0.40,0.55])
#ax.set_ylim([0.75,0.95])
ax.legend(loc='best')

### Examine the performance for dry thunderstorms

In [ ]:
pcp_thrs = 0.1
Xdt_test = X_test.where(X_test.pcp<pcp_thrs).dropna()
ydt_predict_truth = y_test[output_name].where(X_test.pcp<pcp_thrs).dropna().values.ravel()

Dry thunderstorm fraction (i.e., the ratio between dry thunderstorms and all thunderstorms)

In [ ]:
print(y_test.where(y_test.isT>0).where(X_test.pcp<0.1).count()/y_test.where(y_test.isT>0).count())

In [ ]:
ydt_predict_r14, ydt_score0 = R14.predict(Xdt_test['cape'],Xdt_test['pcp'],r14_thrs)

In [ ]:
AUROCC0 = roc_auc_score(y_predict_truth, y_score0)

precision, recall, thresholds = precision_recall_curve(ydt_predict_truth, ydt_score0)
AUPRC0  = auc(recall, precision)

In [ ]:
for imodel in np.arange(1,n_models,1):
    
    exec( 'ydt_predict_rfclf'+str(imodel)+' = rfclf'+str(imodel)+'.predict(Xdt_test[feature_name'+str(imodel)+'])' )
    
    exec( 'ydt_score'+str(imodel)+' = rfclf'+str(imodel)+'.predict_proba(Xdt_test[feature_name'+str(imodel)+'])[:,1]' )
    exec( 'precision, recall, thresholds = precision_recall_curve(ydt_predict_truth, ydt_score'+str(imodel)+')' )
    
    exec( 'AUROCC'+str(imodel)+' = roc_auc_score(ydt_predict_truth, ydt_score'+str(imodel)+')' )
    exec( 'AUPRC'+str(imodel)+' = auc(recall, precision)' )

In [ ]:
colors  = ['k','b','orange','g','r','purple']
fig, ax = plt.subplots()

for imodel in np.arange(0,n_models,1):
    exec( 'fpr, tpr, threshold = roc_curve(ydt_predict_truth, ydt_score'+str(imodel)+')' )
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, c=colors[imodel], label = 'Model '+str(imodel)+' (AUC = %0.2f)'% roc_auc)
    
plt.xlabel('False Alarm Rate')
plt.ylabel('True Positive Rate')
plt.legend(fontsize=12,loc='best')
plt.show()  

In [ ]:
fig, ax = plt.subplots()

for imodel in np.arange(0,n_models,1):
    exec( 'precision, recall, thresholds = precision_recall_curve(ydt_predict_truth, ydt_score'+str(imodel)+')' )
    pr_auc = auc(recall, precision)
    ax.plot(precision, recall, c=colors[imodel], label = 'Model '+str(imodel)+' (AUC = %0.2f)'% pr_auc)
    
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(fontsize=12,loc='best')
plt.show()  

In [ ]:
markers = ['.','v','s','p','*','x','d']
fig, ax = plt.subplots()
for imodel in np.arange(0,n_models,1):
    exec( 'ax.scatter(AUPRC'+str(imodel)+',AUROCC'+str(imodel)+', c=colors[imodel], marker=\''+markers[imodel]+'\',label=\'Model '+str(imodel)+'\')' )

ax.set_title('Model skill')
ax.set_xlabel('Area under PR curve')
ax.set_ylabel('Area under ROC curve')
#ax.set_xlim([0.40,0.55])
#ax.set_ylim([0.75,0.95])
ax.legend(loc='best')